# ELK_Stack 

ELK 는 Elasticsearch, Logstash, Kibana 의 줄임말이고 데이터를 수집/저장하고 시각화에 유용한 어플리케이션 묶음이다.

각 어떤 역할을 담당하고 있는지 말해보자면, 

1. ElasticSeach (ES)

검색 / 분석 엔진으로 데이터를 신속하게 저장, 검색하는 기능을 한다.

Json 기반으로 데이터를 저장하며 NoSQL DB 의 역할을 한다.

HA 를 보장하기 위해 n 개의 노드를 생성할 수 있다. (ES01, ES02 ...)

2. Logstash 

데이터를 수집, 변환하는 기능을 한다.

수집한 데이터를 파싱, 필터링, 변환하여 ES 에 데이터를 put 한다.

regex 의 기능을 담당한다고 생각하면 될 것 같다.

3. Kibana 

ES 에 저장된 데이터를 시각화(대시보드)하는 웹 기반 인터페이스이다.

- - - 

### __ELK_Stack 은 위와 같이 데이터를 수집 / 변환하여 저장하고, 저장된 데이터를 시각화하여 분석하기 위한 Stack__

사실 위의 적힌 내용으로는 그냥 그렇구나 하는 생각밖에 들지 않아 예시를 들어 설명하면 좋을 것 같다.

만약, 프로젝트의 규모가 크지 않아서 log file 의 개수가 적다면 추적 / 관리하는 것에 어려움이 없을 수 있다.

하지만 프로젝트의 규모가 점점 커져서 log file, log 의 개수가 많아지면 추적/ 관리하는 것에 어려움이 없을까 ??

극단적으로 EC2 로 배포한 Sever 가 20 개이고, 그 모든 Server 의 log 가 필요하다면,

아무리 프로젝트의 구성이 완벽하다 하더라도 log 의 추적 / 관리가 어려울 것이다. (ssh login 으로 하나하나??)

하지만 ELK 와 kafka 를 쓰게되면 장애를 허용하며(FA), 고 가용성(HA) 을 유지할 수 있고, log file 의 추적 / 관리가 용이해진다.

그림을 통해 ELK 가 어떻게 데이터를 수집 / 처리 하는지 알아보자.

- - - 
<img src ='https://drive.google.com/uc?id=1WippyR1x2sMZJ_HGpxUDI5_ksKCmFmua'>

위의 그림에서 doc1, doc2, doc3 은 각각 log file 이라고 생각하면 된다.

Server 에 존재하는 log file 들을 logstash 로 가져오고, 파싱, 변환하여 

(stopword 라고 표현된 부분은 logstash 를 통해 파싱, 변환 (문자열 등)을 하는 부분) ES 의 index 에 저장한다.
- - - 

### data 들이 처리되는 과정
기존의 RDB 였다면 

|documnet|content|
|------|-------|
|doc1|The bright ~|
|doc2|it's best ~|
|doc3|under blue ~|

와 같이 저장되어 있었겠지만 

ES 에 저장하면 

|content|doc|
|------|---|
|The|doc1, doc2|
|blue|doc3, doc4|
|best|doc2, doc1|

위와 같이 저장된다.

따라서 만약 계정 관련 로그를 살펴봐야 한다고 했을 때 어디 서버에 있더라? 를 위와 같은 타입으로 확인할 수 있다.

ES 와 RDB 의 이해를 돕기 위한 표는 다음과 같다.

|ES|RDB|
|------|---|
|Index|DB|
|Type|Table|
|Document|row|
|Feild|Column|
|Mapping|Schema|

그리고 ES 는 API 를 지원하기 때문에 config 를 조정하거나 데이터를 요청할 수 있다. 내용은 다음과 같다.

|ES|RDB|
|------|---|
|Get|Select|
|Put|Update|
|Post|Insert|
|Delete|Delete|
- - -
### 처리된 데이터들이 ES 로 보내질 때의 과정
<img src ='https://drive.google.com/uc?id=1FNuKGTWmISPxW2CnM0nGPUgQ2_jwYQZc'>

Indexing 된 데이터들은 ES 클러스터 내부에서 각 노드에 shard 로 분산 저장된다.

데이터를 적절한 샤드로 라우팅해서 다량의 데이터들을 처리하고 성능을 높일 수 있다.

그림과 같이 ES 클러스터를 n 개로 구성하게 되면 HA 에 많은 도움이 되는데, 

어느 한 노드에 장애가 발생하더라도 primary shard 를 replica shard 로 만들어 다른 노드에 저장하여 

ES 노드의 장애에 대처하게 할 수 있다.

- - -
### 전체적인 흐름 정리

<img src = 'https://drive.google.com/uc?id=1nnV18w7jxidQDmGJLP_i6Y1Eu7C6GDXb'>

각 서버의 log 를 수집하여 파싱, 변환하고 ES (NoSQL) 에 저장 후 시각화, 분석을 진행한다.

- - -
### 추가 사항

필자는 Queue 자리에 Kafka 를 사용하여 logstash 와 kafka 를 연동 후 Server Event Stream 에 대한 log 들을 수집하고 시각화를 해보려한다.

그 이유는 프로젝트의 규모가 커졌을 때 logstash 로 각 서버의 log 를 일괄 관리할 수 있지만, 

logstash 에 장애 발생 시 데이터 손실을 막을 방법이 없어져 그에 대한 해결책으로 kafka 를 사용하고,

Zookeeper 로 kafka 의 HA 를 보장하며 데이터를 버퍼링하기 때문에 데이터 손실의 문제를 최소화 하려고 한다.

In [13]:
from elasticsearch import Elasticsearch


USER = 'User'
PASSW = 'Password'
API_KEY = "Nnc5LXk0b0J6eXNMTG1wbmNna286SmhPS1FKc1NUSFdNeWJNc21vOTZIdw=="

es = Elasticsearch("http://localhost:9200", basic_auth = (USER, PASSW))



es.index(index = "my_files_index", document = {
    "path": "/home/owen/hadoop/LICENSE.txt",
    "content": open("/home/owen/hadoop/LICENSE.txt").read()
})


/tmp/ipykernel_17770/3970241107.py:9: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index = "my_files_index", body = {


ObjectApiResponse({'_index': 'my_files_index', '_id': '9A_7y4oBzysLLmpndAl8', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

# Elastic search 는 자체적으로 api 를 제공함. 

__postman 으로 rest api 테스트__

<img src = "https://drive.google.com/uc?id=1kUuKITIfX0L7QNbH21t7pR-mJrqFY6t2">